In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as req
import re
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth',None)
import re

## Testing

In [2]:
nb_page = 1
url = f'https://www.cvedetails.com/cwe-definitions/{nb_page}/cwelist.html?order=2&trc=668&sha=0427874cc45423ccb6974ee25935fbfceac76fcb'
html = req.get(url)
soup = bs(html.content,'lxml')
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html>
<head>
<meta content="width=900" name="viewport"/>
<title>CWE Definitions list and vulnerabilities for CWE entries</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="CWE definitions list. Common Weakness Enumeration (CWE™) is a list of software weaknesses.  " name="description"/>
<meta content="security, cve, nvd, security vulnerability, vulnerability, exploit, cvss, cwe, information security, vulnerability database" name="keywords"/>
<link href="/cvedetails.css" rel="stylesheet" type="text/css"/>
<meta content="349381870180-eaoug67mdrnbmuink9n43t3b8silo5h0.apps.googleusercontent.com" name="google-signin-client_id"/>
<script src="/cvedetails.js" type="text/javascript"></script>
</head>
<body>
<div id="topcontainer">
<div id="topleft">
<a href="/" title="Go to cvedetails.com homepage"><img alt="CVEdetails.com the ultimate security vulnerability data 

In [4]:
# Extract the range number of pages
pagingb = soup.find('div',id="pagingb")
pages_nums = pagingb.findAll('a')
pn_list = []
for p in pages_nums:
    pn_list.append(p.text)
print(pn_list)
print('total pages available in this year is : ' ,pn_list[-1])
print(range(len(pn_list)))
print(len(pn_list))

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
total pages available in this year is :  14
range(0, 14)
14


In [5]:
# Extract Header
table = soup.find('table',class_='listtable')
headers = [header.text.strip() for header in table.find_all("th")]
headers

['CWE Number', 'Name', 'Number Of Related Vulnerabilities']

In [44]:
# Extract cwe_number links
table = soup.find('table',class_="listtable")
tds = table.find_all('td')
td_tags_without_attrs = table.find_all(lambda tag: tag.name == "td" and not tag.attrs) # Find td tags without any attributes
for td in td_tags_without_attrs:
    if td.find('a') is not None:
        print(td.find('a')['href'],'\n')
        

//www.cvedetails.com/cwe-details/79/Failure-to-Preserve-Web-Page-Structure-039-Cross-site-Scr.html 

//www.cvedetails.com/cwe-details/119/Failure-to-Constrain-Operations-within-the-Bounds-of-a-Memor.html 

//www.cvedetails.com/cwe-details/89/Improper-Sanitization-of-Special-Elements-used-in-an-SQL-Com.html 

//www.cvedetails.com/cwe-details/20/Improper-Input-Validation.html 

//www.cvedetails.com/cwe-details/787/Out-of-bounds-Write.html 

//www.cvedetails.com/cwe-details/200/Information-Exposure.html 

//www.cvedetails.com/cwe-details/22/Improper-Limitation-of-a-Pathname-to-a-Restricted-Directory-.html 

//www.cvedetails.com/cwe-details/125/Out-of-bounds-Read.html 

//www.cvedetails.com/cwe-details/416/Use-After-Free.html 

//www.cvedetails.com/cwe-details/287/Improper-Authentication.html 

//www.cvedetails.com/cwe-details/94/Failure-to-Control-Generation-of-Code-039-Code-Injection-.html 

//www.cvedetails.com/cwe-details/78/Improper-Sanitization-of-Special-Elements-used-in-an-OS-Comm.

In [121]:
# Extract description and background_details from https://www.cvedetails.com/cwe-details/79
url = 'https://www.cvedetails.com/cwe-details/79'
html = req.get(url)
soup = bs(html.content,'lxml')
table = soup.find('table',class_="details")
tds = table.find_all('td',style="text-align: justify; ")

description = re.sub(r'[\n\t]', ' ', tds[0].text)
background_details = re.sub(r'[\n\t]', ' ', tds[1].text)

print('Description:',description , '\n\nBackground Details:' , background_details)

Description: The software does not sufficiently validate, filter, escape,and/or encode user-controllable input before it is placed in output that is usedas a web page that is served to other users.Cross-site scripting (XSS) vulnerabilities occur when: 

Background Details: The same origin policy states that browsers should limit the resourcesaccessible to scripts running on a given web site , or "origin", to theresources associated with that web site on the client-side, and not theclient-side resources of any other sites or "origins". The goal is toprevent one site from being able to modify or read the contents of anunrelated site. Since the World Wide Web involves interactions between manysites, this policy is important for browsers to enforce.


## CWE Scraper

In [6]:
def cwe_scraper():
    
    # Extract the range number of pages
    url_for_pages = 'https://www.cvedetails.com/cwe-definitions/2/cwelist.html?order=2&trc=668&sha=0427874cc45423ccb6974ee25935fbfceac76fcb'
    html_for_pages = req.get(url_for_pages)
    soup_for_pages = bs(html_for_pages.content,'lxml')
    pagingb = soup_for_pages.find('div',id="pagingb")
    pages_nums = pagingb.findAll('a')
    pn_list = []
    for p in pages_nums:
        pn_list.append(p.text)
    nb_pages = pn_list[-1]

    # scrape all pages
    rows = []
    for page in range(1,int(nb_pages)+1):
        url = f'https://www.cvedetails.com/cwe-definitions/{page}/cwelist.html?order=2&trc=668&sha=0427874cc45423ccb6974ee25935fbfceac76fcb'
        html = req.get(url)
        soup = bs(html.content,'lxml')
        table = soup.find('table',class_='listtable')
        headers = [header.text.strip() for header in table.find_all("th")]
        for row in table.find_all("tr")[1:]:
            rows.append([data.text.strip() for data in row.find_all("td")])

    return pd.DataFrame(rows,columns=headers)

In [7]:
df = cwe_scraper()
df.to_csv('CWE_definitions.csv')
pd.read_csv('CWE_definitions.csv',)

,Unnamed: 0,CWE Number,Name,Number Of Related Vulnerabilities
0,0,79,Failure to Preserve Web Page Structure ('Cross...,22413.0
1,1,119,Failure to Constrain Operations within the Bou...,11915.0
2,2,89,Improper Sanitization of Special Elements used...,9554.0
3,3,20,Improper Input Validation,9380.0
4,4,787,Out-of-bounds Write,7611.0
...,...,...,...,...
663,663,82,Improper Sanitization of Script in Attributes ...,NaN
664,664,85,Doubled Character XSS Manipulations,NaN
665,665,9,J2EE Misconfiguration: Weak Access Permissions...,NaN
666,666,92,DEPRECATED: Improper Sanitization of Custom Sp...,NaN


## Scrape each CWE
- https://www.cvedetails.com/cwe-details/613
- http://cwe.mitre.org/data/definitions/613.html

In [79]:
df1 = pd.read_csv('CWE_definitions.csv')
cwe_details_links=[]
cwe_definition_links=[]

for cwe in df1['CWE Number']:
    link1 = 'https://www.cvedetails.com/cwe-details/'+str(cwe)
    link2 = 'http://cwe.mitre.org/data/definitions/'+str(cwe)
    cwe_details_links.append(link1)
    cwe_definition_links.append(link2)
    
df1['CWE Details'] = cwe_details_links
df1['CWE Definition'] = cwe_definition_links

df1

,Unnamed: 0,CWE Number,Name,Number Of Related Vulnerabilities,CWE Details,CWE Definition
0,0,79,Failure to Preserve Web Page Structure ('Cross-site Scripting'),22413.0,https://www.cvedetails.com/cwe-details/79,http://cwe.mitre.org/data/definitions/79
1,1,119,Failure to Constrain Operations within the Bounds of a Memory Buffer,11915.0,https://www.cvedetails.com/cwe-details/119,http://cwe.mitre.org/data/definitions/119
2,2,89,Improper Sanitization of Special Elements used in an SQL Command ('SQL Injection'),9554.0,https://www.cvedetails.com/cwe-details/89,http://cwe.mitre.org/data/definitions/89
3,3,20,Improper Input Validation,9380.0,https://www.cvedetails.com/cwe-details/20,http://cwe.mitre.org/data/definitions/20
4,4,787,Out-of-bounds Write,7611.0,https://www.cvedetails.com/cwe-details/787,http://cwe.mitre.org/data/definitions/787
...,...,...,...,...,...,...
663,663,82,Improper Sanitization of Script in Attributes of IMG Tags in a Web Page,NaN,https://www.cvedetails.com/cwe-details/82,http://cwe.mitre.org/data/definitions/82
664,664,85,Doubled Character XSS Manipulations,NaN,https://www.cvedetails.com/cwe-details/85,http://cwe.mitre.org/data/definitions/85
665,665,9,J2EE Misconfiguration: Weak Access Permissions for EJB Methods,NaN,https://www.cvedetails.com/cwe-details/9,http://cwe.mitre.org/data/definitions/9
666,666,92,DEPRECATED: Improper Sanitization of Custom Special Characters,NaN,https://www.cvedetails.com/cwe-details/92,http://cwe.mitre.org/data/definitions/92


In [127]:
%%time
description=[]
background_details=[]

for link in df1['CWE Details']:
    html = req.get(link)
    soup = bs(html.content,'lxml')
    table = soup.find('table',class_="details")
    tds = table.find_all('td',style="text-align: justify; ")

    description.append(re.sub(r'[\n\t]', ' ', tds[0].text))
    background_details.append(re.sub(r'[\n\t]', ' ', tds[1].text))


CPU times: total: 56.4 s
Wall time: 12min 57s


In [132]:
df1['Description'] = description
df1['Background Details'] = background_details

In [139]:
df1.to_csv('CWE Definitions All.csv',index=False)

In [140]:
pd.read_csv('CWE Definitions All.csv')

,Unnamed: 0,CWE Number,Name,Number Of Related Vulnerabilities,CWE Details,CWE Definition,Description,Background Details
0,0,79,Failure to Preserve Web Page Structure ('Cross-site Scripting'),22413.0,https://www.cvedetails.com/cwe-details/79,http://cwe.mitre.org/data/definitions/79,"The software does not sufficiently validate, filter, escape, and/or encode user-controllable input before it is placed in output that is used as a web page that is served to other users.Cross-site scripting (XSS) vulnerabilities occur when:","The same origin policy states that browsers should limit the resources accessible to scripts running on a given web site , or ""origin"", to the resources associated with that web site on the client-side, and not the client-side resources of any other sites or ""origins"". The goal is to prevent one site from being able to modify or read the contents of an unrelated site. Since the World Wide Web involves interactions between many sites, this policy is important for browsers to enforce."
1,1,119,Failure to Constrain Operations within the Bounds of a Memory Buffer,11915.0,https://www.cvedetails.com/cwe-details/119,http://cwe.mitre.org/data/definitions/119,"The software performs operations on a memory buffer, but it can read from or write to a memory location that is outside of the intended boundary of the buffer.Certain languages allow direct addressing of memory locations and do not automatically ensure that these locations are valid for the memory buffer that is being referenced. This can cause read or write operations to be performed on memory locations that may be associated with other variables, data structures, or internal program data.",
2,2,89,Improper Sanitization of Special Elements used in an SQL Command ('SQL Injection'),9554.0,https://www.cvedetails.com/cwe-details/89,http://cwe.mitre.org/data/definitions/89,"The software constructs all or part of an SQL command using externally-influenced input from an upstream component, but it does not sanitize or incorrectly sanitizes special elements that could modify the intended SQL command when it is sent to a downstream component.Without sufficient removal or quoting of SQL syntax in user-controllable inputs, the generated SQL query can cause those inputs to be interpreted as SQL instead of ordinary user data. This can be used to alter query logic to bypass security checks, or to insert additional statements that modify the back-end database, possibly including execution of system commands.",
3,3,20,Improper Input Validation,9380.0,https://www.cvedetails.com/cwe-details/20,http://cwe.mitre.org/data/definitions/20,"The product does not validate or incorrectly validates input that can affect the control flow or data flow of a program.When software fails to validate input properly, an attacker is able to craft the input in a form that is not expected by the rest of the application. This will lead to parts of the system receiving unintended input, which may result in altered control flow, arbitrary control of a resource, or arbitrary code execution.",
4,4,787,Out-of-bounds Write,7611.0,https://www.cvedetails.com/cwe-details/787,http://cwe.mitre.org/data/definitions/787,"The software writes data past the end, or before the beginning, of the intended buffer.This typically occurs when the pointer or its index is incremented or decremented to a position beyond the bounds of the buffer or when pointer arithmetic results in a position outside of the valid memory location to name a few. This may result in corruption of sensitive information, a crash, or code execution among other things.",
...,...,...,...,...,...,...,...,...
663,663,82,Improper Sanitization of Script in Attributes of IMG Tags in a Web Page,NaN,https://www.cvedetails.com/cwe-details/82,http://cwe.mitre.org/data/definitions/82,"The web application does not filter or incorrectly filters scripting elements within attributes of HTML IMG tags, such as the src attribute.Attackers can embed XSS exploits into 